In [1]:
import pandas as pd

data = pd.read_csv('data/annual_KY_WI_4_class_balanced.csv')
data.head()

,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Class
0,Wisconsin,Marshfield,42095,Vernal,42243,239,1.19,148,1,2883.00,453.6,16.84000,11.08000,22.62000,0.070000,1
1,Kentucky,Lexington,4/12/2018,NF11ALF006,8/20/2018,232,1.30,130,1,1856.67,440.3,23.38475,18.56275,28.45425,0.272962,1
2,Wisconsin,Marshfield,42095,FF42.A2,42243,239,1.32,148,1,2883.00,453.6,16.84000,11.08000,22.62000,0.070000,1
3,Kentucky,Lexington,4/12/2018,BYS5028,8/20/2018,232,1.34,130,1,1856.67,440.3,23.38475,18.56275,28.45425,0.272962,1
4,Wisconsin,Marshfield,42095,55Q27,42243,239,1.35,148,1,2883.00,453.6,16.84000,11.08000,22.62000,0.070000,1


In [2]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

/home/myid/jmvance/.local/lib/python3.6/site-packages/sklearn/mixture/_base.py:148: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  random_state=random_state).fit(X).labels_


In [3]:
new_data = model.sample(1000)

In [4]:
new_data.to_csv('data/CTG_synth1k_0723_KY_WI_knn_4cls_v2.csv')
new_data.head()

,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%),Class
0,Kentucky,Lexington,4/5/2017,MSSUNSTRA-C11*,42613,244,4.50,469,343,3330.49,851.6,18.202331,18.878327,11.117364,0.123548,3
1,Wisconsin,Lexington,8/9/2012,LH 9700,9/13/2018,227,4.89,251,107,3284.31,749.3,8.532075,18.878327,10.920000,0.246284,1
2,Kentucky,Lexington,4/5/2016,AFX 103009,41886,256,3.44,460,1,2761.85,742.5,17.783595,1.968787,14.861641,0.139938,1
3,Wisconsin,Lexington,42095,MSSUNSTRA-D11*,42251,253,2.12,619,402,3595.13,984.5,21.342054,14.340357,11.036934,0.112180,1
4,Wisconsin,Marshfield,41730,Arc,9/16/2015,247,1.48,531,402,4533.46,872.8,15.651468,-0.530000,27.805417,0.316583,1


In [5]:
# get aggregate data
targetDataLoc = 'data/annual_GA_4_class_1varPerYear_unordered.csv'
#aggDataLoc = 'data/synth1_GA_only_063022.csv'

aggDf = new_data #pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)

In [6]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVC  # support vector machines classification
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsClassifier # k-nearest neightbors for classification
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier # adaboost for classification
from sklearn.neural_network import MLPClassifier
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [7]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
               "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
                 "Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Class"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
test_xDf = targetDf[xColumnsToKeep]
yDf = aggDf[yColumnsToKeep]
test_yDf = targetDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = test_xDf.reset_index(drop=True)
test_yDf = test_yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [8]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["gini"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svcParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

logRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['gini'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [#(RandomForestClassifier(), rfParamGrid, "rfTup.pkl"),
              (KNeighborsClassifier(), knnParamGrid, "knnTup.pkl"),
              (SVC(), svcParamGrid, "svrTup.pkl"),
             #(MLPClassifier(), nnParamGrid, "nnTup.pkl")]#,
             (LogisticRegression(), logRegParamGrid, "linRegTup.pkl")]#,
             #(BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             #(DecisionTreeClassifier(), dtParamGrid, "dtTup.pkl")]

N = 10
workingDir = 'working_dir'
numFeatures = 8 # 11

In [9]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

model:  knnTup
Avg MAE:  1590.0
Avg R squared:  -0.717
Best MAE:  1280.0
Best R squared:  -0.591
Parameters of the best model:  {'model__leaf_size': 5, 'model__n_neighbors': 10, 'model__weights': 'uniform'}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
Accuracy:  0.52
f1 score:  0.25800705589374656
mcc:  0.08476181134583922
 
test results on our test data: 
results:
MAE:  0.75
R sq:  0.19999999999999996
accuracy score for DA/TDA:  0.375
model:  svrTup
Avg MAE:  1542.0
Avg R squared:  -0.689
Best MAE:  1180.0
Best R squared:  -0.469
Parameters of the best model:  {'model__C': 0.1, 'model__degree': 4, 'model__gamma': 'scale', 'model__kernel': 'poly'}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Tem